In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
%matplotlib inline

In [2]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights_pruned_total.csv', low_memory=False)

In [7]:
anc_weather = pd.read_csv('PANC_201501010000_201601010000.csv', skiprows=5)

In [9]:
anc_weather.head()

,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,...,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar
0,PANC,2015-01-01 00:53,-150.0261,61.1697,39.92,32.00,73.08,170.00,6.0,0.00,...,FEW,BKN,OVC,M,2500.00,5500.00,7000.00,M,-RA,PANC 010053Z 17006KT 10SM -RA FEW025 BKN055 OV...
1,PANC,2015-01-01 01:53,-150.0261,61.1697,39.02,32.00,75.69,150.00,6.0,0.00,...,SCT,BKN,BKN,M,3500.00,5500.00,7000.00,M,M,PANC 010153Z 15006KT 10SM SCT035 BKN055 BKN070...
2,PANC,2015-01-01 02:53,-150.0261,61.1697,39.92,32.00,73.08,200.00,4.0,0.00,...,SCT,BKN,OVC,M,3000.00,5500.00,7000.00,M,-RA,PANC 010253Z 20004KT 9SM -RA SCT030 BKN055 OVC...
3,PANC,2015-01-01 03:14,-150.0261,61.1697,39.20,33.80,80.79,160.00,5.0,0.02,...,BKN,OVC,M,M,2500.00,5000.00,M,M,-RA,PANC 010314Z 16005KT 10SM -RA BKN025 OVC050 04...
4,PANC,2015-01-01 03:53,-150.0261,61.1697,39.02,33.08,79.05,150.00,6.0,0.03,...,BKN,OVC,M,M,2700.00,4000.00,M,M,-RA,PANC 010353Z 15006KT 10SM -RA BKN027 OVC040 04...


In [11]:
anc_weather.loc[anc_weather['station'] == 'PANC', 'station'] = 'ANC'

In [12]:
anc_weather.head()

,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,...,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar
0,ANC,2015-01-01 00:53,-150.0261,61.1697,39.92,32.00,73.08,170.00,6.0,0.00,...,FEW,BKN,OVC,M,2500.00,5500.00,7000.00,M,-RA,PANC 010053Z 17006KT 10SM -RA FEW025 BKN055 OV...
1,ANC,2015-01-01 01:53,-150.0261,61.1697,39.02,32.00,75.69,150.00,6.0,0.00,...,SCT,BKN,BKN,M,3500.00,5500.00,7000.00,M,M,PANC 010153Z 15006KT 10SM SCT035 BKN055 BKN070...
2,ANC,2015-01-01 02:53,-150.0261,61.1697,39.92,32.00,73.08,200.00,4.0,0.00,...,SCT,BKN,OVC,M,3000.00,5500.00,7000.00,M,-RA,PANC 010253Z 20004KT 9SM -RA SCT030 BKN055 OVC...
3,ANC,2015-01-01 03:14,-150.0261,61.1697,39.20,33.80,80.79,160.00,5.0,0.02,...,BKN,OVC,M,M,2500.00,5000.00,M,M,-RA,PANC 010314Z 16005KT 10SM -RA BKN025 OVC050 04...
4,ANC,2015-01-01 03:53,-150.0261,61.1697,39.02,33.08,79.05,150.00,6.0,0.03,...,BKN,OVC,M,M,2700.00,4000.00,M,M,-RA,PANC 010353Z 15006KT 10SM -RA BKN027 OVC040 04...


In [57]:
test_dt = '2015-05-06 03:35'

#test_dt_a = datetime.strptime(test_dt, format='%Y-%m-%d %H:%M')
def getClosestWeather(date_str, airport_code):
    test_dt = datetime.datetime(*(time.strptime(date_str,'%Y-%m-%d %H:%M')[0:6]))

    start_dt = test_dt - datetime.timedelta(hours=1)
    end_dt = test_dt + datetime.timedelta(hours=1)

    start_dt_str = start_dt.strftime('%Y-%m-%d %H:%M')
    end_dt_str = end_dt.strftime('%Y-%m-%d %H:%M')

    mask = (anc_weather['valid'] > start_dt_str) & (anc_weather['valid'] <= end_dt_str)

    close_dts = anc_weather.loc[mask]

    str_dates = close_dts['valid'].tolist()
    dates = []
    for date in str_dates:
        dates.append(datetime.datetime(*(time.strptime(date,'%Y-%m-%d %H:%M')[0:6])))

    final_dt = min(dates, key=lambda d: abs(d - test_dt))
    return final_dt

print(getClosestWeather(test_dt, 0))

2015-05-06 02:35:00
2015-05-06 04:35:00
[datetime.datetime(2015, 5, 6, 2, 53), datetime.datetime(2015, 5, 6, 3, 53)]
2015-05-06 03:53:00


In [55]:
new_anc_weather = anc_weather[['station', 'valid', 'tmpf', ' dwpf', ' relh', ' sknt', ' p01i', ' alti', ' vsby', ' gust', \
                                            ' skyc1', ' presentwx']]
new_anc_weather

Index(['station', 'valid', 'lon', 'lat', 'tmpf', ' dwpf', ' relh', ' drct',
       ' sknt', ' p01i', ' alti', ' mslp', ' vsby', ' gust', ' skyc1',
       ' skyc2', ' skyc3', ' skyc4', ' skyl1', ' skyl2', ' skyl3', ' skyl4',
       ' presentwx', ' metar'],
      dtype='object')


,station,valid,tmpf,dwpf,relh,sknt,p01i,alti,vsby,gust,skyc1,presentwx
0,ANC,2015-01-01 00:53,39.92,32.00,73.08,6.0,0.00,30.27,10.00,M,FEW,-RA
1,ANC,2015-01-01 01:53,39.02,32.00,75.69,6.0,0.00,30.27,10.00,M,SCT,M
2,ANC,2015-01-01 02:53,39.92,32.00,73.08,4.0,0.00,30.27,9.00,M,SCT,-RA
3,ANC,2015-01-01 03:14,39.20,33.80,80.79,5.0,0.02,30.26,10.00,M,BKN,-RA
4,ANC,2015-01-01 03:53,39.02,33.08,79.05,6.0,0.03,30.26,10.00,M,BKN,-RA
5,ANC,2015-01-01 04:53,37.94,33.08,82.46,8.0,0.04,30.24,6.00,M,BKN,-RA
6,ANC,2015-01-01 05:53,37.04,33.08,85.43,4.0,0.00,30.23,10.00,M,FEW,-RA
7,ANC,2015-01-01 06:07,37.40,33.80,86.69,3.0,0.00,30.22,10.00,M,SCT,M
8,ANC,2015-01-01 06:53,37.04,33.08,85.43,4.0,0.00,30.21,10.00,M,FEW,M
9,ANC,2015-01-01 07:53,35.96,32.00,85.37,6.0,0.00,30.20,10.00,M,SCT,M


In [56]:
flights.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DIVERTED,CANCELLED
0,1,1,4,AS,ANC,SEA,5,1448,430,-22.0,0,0
1,1,1,4,AA,LAX,PBI,10,2330,750,-9.0,0,0
2,1,1,4,US,SFO,CLT,20,2296,806,5.0,0,0
3,1,1,4,AA,LAX,MIA,20,2342,805,-9.0,0,0
4,1,1,4,AS,SEA,ANC,25,1448,320,-21.0,0,0


In [62]:
anc_flights = flights.loc[flights['ORIGIN_AIRPORT'] == 'ANC',]
anc_flights.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DIVERTED,CANCELLED
0,1,1,4,AS,ANC,SEA,5,1448,430,-22.0,0,0
15,1,1,4,AS,ANC,SEA,45,1448,509,-14.0,0,0
16,1,1,4,DL,ANC,SEA,45,1448,515,-24.0,0,0
18,1,1,4,AS,ANC,PDX,50,1542,525,-18.0,0,0
32,1,1,4,AS,ANC,SEA,135,1448,600,NaN,0,1


In [ ]:
#pd.merge(anc_flights, anc_weather, on='subject_id', how='inner')
anc_flights.merge(anc_weather, how='inner', left_on=, right_on=None)